In [21]:
pip install git+https://github.com/openai/swarm.git

  Cloning https://github.com/openai/swarm.git to /private/var/folders/n5/3gq4rjw557n363fmnygxh7280000gn/T/pip-req-build-w88ma5ut
  Running command git clone --filter=blob:none --quiet https://github.com/openai/swarm.git /private/var/folders/n5/3gq4rjw557n363fmnygxh7280000gn/T/pip-req-build-w88ma5ut
  Resolved https://github.com/openai/swarm.git to commit 9db581cecaacea0d46a933d6453c312b034dbf47
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [35]:
from swarm import Swarm, Agent
import openai
import os
from dotenv import load_dotenv

load_dotenv()  # Load environment variables from .env

# Set up OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Initialize Swarm client
client = Swarm()

# Define Agent 2: ILAE Score Calculator
def agent_2_instructions(context_variables):
    return f"""
You are a medical expert specializing in epilepsy. Your task is to calculate the ILAE score based on the provided clinical information. Use the ILAE Outcome Scale criteria below to determine the correct score. Provide detailed reasoning for each entity influencing the score.

**ILAE Outcome Scale:**
- **Class 1**: Completely seizure free; no auras
- **Class 2**: Only auras; no other seizures
- **Class 3**: 1 to 3 seizure days per year; ± auras
- **Class 4**: 4 seizure days per year to 50% reduction of baseline seizure days; ± auras
- **Class 5**: Less than 50% reduction of baseline seizure days; ± auras
- **Class 6**: More than 100% increase of baseline seizure days; ± auras

**Extracted Entities:**
1. Presence of seizure freedom: {context_variables.get('presence_of_seizure_freedom', "I don't know")}
2. Presence of auras: {context_variables.get('presence_of_auras', "I don't know")}
3. Baseline seizure days (pre-treatment): {context_variables.get('baseline_seizure_days', "I don't know")}
4. Seizure days per year (post-treatment): {context_variables.get('seizure_days_per_year', "I don't know")}
5. Percent reduction in seizure days: {context_variables.get('percent_reduction_in_seizure_days', "I don't know")}

Calculate the ILAE score using this information, and provide detailed reasoning for each entity influencing the score.
    """

agent_2 = Agent(
    name="ILAE Score Calculator",
    instructions=agent_2_instructions
)

# Define the function to hand off to Agent 2
def handoff_to_agent_2(
    presence_of_seizure_freedom: str,
    presence_of_auras: str,
    baseline_seizure_days: str,
    seizure_days_per_year: str,
    context_variables
):
    context_variables.update({
        "presence_of_seizure_freedom": presence_of_seizure_freedom,
        "presence_of_auras": presence_of_auras,
        "baseline_seizure_days": baseline_seizure_days,
        "seizure_days_per_year": seizure_days_per_year,
    })
    
    # Convert baseline seizure days and seizure days per year to floats
    try:
        baseline_seizure_days_value = float(baseline_seizure_days)
        seizure_days_per_year_value = float(seizure_days_per_year)
        
        # Check for zero baseline seizure days to avoid division by zero
        if baseline_seizure_days_value == 0:
            percent_reduction = "Cannot calculate (baseline seizure days is zero)"
        else:
            percent_reduction_value = ((baseline_seizure_days_value - seizure_days_per_year_value) / baseline_seizure_days_value) * 100
            percent_reduction = f"{percent_reduction_value:.2f}%"
    except ValueError:
        percent_reduction = "Cannot calculate (invalid numbers)"
    
    context_variables["percent_reduction_in_seizure_days"] = percent_reduction
    
    # Return the agent_2 instance and updated context_variables
    return agent_2, context_variables

# Define Agent 1: Clinical Information Extractor
def agent_1_instructions(context_variables):
    return f"""
You are a clinical information extractor. Your task is to extract the following entities from the clinical note provided:

1. Presence of seizure freedom.
2. Presence of auras.
3. Baseline seizure days (pre-treatment).
4. Seizure days per year (post-treatment).

Please ensure that:
- **Baseline seizure days** refers to the number of seizure days per year before any treatment or surgery (pre-treatment).
- **Seizure days per year** refers to the number of seizure days per year after treatment or surgery (post-treatment).

Clinical Note:
{context_variables.get('clinical_note', '')}

Use the clinical note to extract these entities accurately. If any entity information is missing or not explicitly stated in the clinical note, respond with "I don't know" for that particular entity. Once you have extracted the information, call the function 'handoff_to_agent_2' with the extracted entities as arguments.
    """

agent_1 = Agent(
    name="Clinical Information Extractor",
    instructions=agent_1_instructions,
    functions=[handoff_to_agent_2]
)

# Define the clinical note
clinical_note = """
Patient is a 35-year-old male with a history of epilepsy. He used to experience seizures approximately twice a month before treatment. Over the past year after surgery, his seizures have reduced to once every three months. He reports occasional auras before the seizures occur. No complete seizure freedom has been achieved yet.
"""

# Set up context variables
context_variables = {
    "clinical_note": clinical_note
}

# Set up initial messages
messages = [
    {"role": "user", "content": "Please process the clinical note."}
]

# Run the Swarm client
response = client.run(
    agent=agent_1,
    messages=messages,
    context_variables=context_variables,
    max_turns=5  # Optional: Prevent infinite loops
)

# Print the final response
for message in response.messages:
    sender = message.get('sender', 'Agent')
    content = message['content']
    print(f"{sender}: {content}")


Clinical Information Extractor: None
Agent: (Agent(name='ILAE Score Calculator', model='gpt-4o', instructions=<function agent_2_instructions at 0x1412a1c60>, functions=[], tool_choice=None, parallel_tool_calls=True), {'clinical_note': '\nPatient is a 35-year-old male with a history of epilepsy. He used to experience seizures approximately twice a month before treatment. Over the past year after surgery, his seizures have reduced to once every three months. He reports occasional auras before the seizures occur. No complete seizure freedom has been achieved yet.\n', 'presence_of_seizure_freedom': 'No', 'presence_of_auras': 'Yes', 'baseline_seizure_days': '24', 'seizure_days_per_year': '4', 'percent_reduction_in_seizure_days': '83.33%'})
Clinical Information Extractor: The clinical note has been processed with the following extracted entities:

- Presence of seizure freedom: No
- Presence of auras: Yes
- Baseline seizure days (pre-treatment): 24
- Seizure days per year (post-treatment): 4